In [81]:
import numpy as np
import struct
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import pandas as pd
import seaborn as sns
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from collections import Counter

# Random Prototyping

In [50]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Convert dataset to DataLoader (load all at once)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=len(train_set), shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=len(test_set), shuffle=False)

# Extract train and test data
train_data, train_labels = next(iter(train_loader))
test_data, test_labels = next(iter(test_loader))

# Flatten images from (N, 1, 28, 28) -> (N, 784)
train_data = train_data.view(train_data.shape[0], -1).to(device)  # (60000, 784)
test_data = test_data.view(test_data.shape[0], -1).to(device)  # (10000, 784)

train_labels = train_labels.to(device)
test_labels = test_labels.to(device)

Using device: cpu


In [39]:
num_subsets = np.array([10000, 5000, 1000])

In [62]:
def compute_accuracy(test_data, prototype_data, prototype_labels, k=1):
    # Compute full pairwise distance matrix in one go
    print("Computing full distance matrix...")
    distances = torch.cdist(test_data, prototype_data)  # Shape: (10000, 60000)

    # Get indices of k nearest neighbors
    k_indices = torch.topk(distances, k, largest=False).indices  # Shape: (10000, k)

    # Retrieve the k nearest labels
    k_labels = prototype_labels[k_indices]  # Shape: (10000, k)

    # Majority voting for prediction
    pred_labels = torch.mode(k_labels, dim=1).values  # Shape: (10000,)

    # Compute accuracy
    accuracy = (pred_labels == test_labels).float().mean().item()
    print(f'k-NN accuracy on full test set (no batching): {accuracy:.4f}')
    return accuracy

In [63]:
accuracy_dict = {}
for subset in num_subsets:
    accuracy_list = []
    for _ in range(30):
        random_indicies = torch.randperm(train_data.shape[0])[:subset]
        prototype_train_data = train_data[random_indicies]
        prototype_train_labels = train_labels[random_indicies]
        accuracy = compute_accuracy(test_data, prototype_train_data, prototype_train_labels)
        accuracy_list.append(accuracy)
    accuracy_dict[subset] = accuracy_list

Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9517
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9477
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9451
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9489
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9489
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9499
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9478
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9493
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9488
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9475
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9503
Computing full distance matrix...
k-NN accu

In [67]:
accuracy_df = pd.DataFrame(accuracy_dict)

In [68]:
accuracy_df.mean()

10000    0.948923
5000     0.935643
1000     0.883680
dtype: float64

In [70]:
accuracy_df.std(ddof=1)

10000    0.001785
5000     0.001749
1000     0.004651
dtype: float64

# K Means

## K Means

### Base Functions

In [104]:
# Perform K-Means clustering
def kmeans(X, k, num_iters=100, tol=1e-4):
    """
    Runs K-Means clustering on tensor X.
    
    Args:
    - X (Tensor): Data points of shape (N, D)
    - k (int): Number of clusters
    - num_iters (int): Maximum number of iterations
    - tol (float): Tolerance for convergence
    
    Returns:
    - cluster_assignments (Tensor): Cluster labels for each point
    - centroids (Tensor): Final cluster centroids
    """
    N, D = X.shape
    device = X.device

    # 1️⃣ Randomly initialize centroids
    indices = torch.randperm(N)[:k]  
    centroids = X[indices]  # (k, D)

    for i in range(num_iters):
        # 2️⃣ Compute distances to centroids
        distances = torch.cdist(X, centroids)  # (N, k)

        # 3️⃣ Assign each point to the nearest centroid
        cluster_assignments = torch.argmin(distances, dim=1)  # (N,)

        # 4️⃣ Compute new centroids
        new_centroids = torch.stack([X[cluster_assignments == c].mean(dim=0) for c in range(k)])

        # 5️⃣ Check for convergence
        if torch.allclose(new_centroids, centroids, atol=tol):
            print(f'Converged at iteration {i}')
            break
        centroids = new_centroids

    return cluster_assignments, centroids

# Function to assign labels to centroids
def assign_labels(cluster_labels, y_true, k):
    """
    Assigns a label to each K-Means cluster using majority voting.
    
    Args:
    - cluster_labels (Tensor): Cluster assignments for each point
    - y_true (Tensor): True MNIST labels
    - k (int): Number of clusters

    Returns:
    - cluster_to_label (list): List where index `i` corresponds to cluster `i`'s assigned label
    """
    cluster_to_label = [-1] * k  # Initialize list with -1 for empty clusters

    for cluster in range(k):
        # Get all true labels for this cluster
        cluster_indices = (cluster_labels == cluster).nonzero(as_tuple=True)[0]
        true_labels = y_true[cluster_indices]

        # Find the most common label in this cluster
        if len(true_labels) > 0:
            most_common_label = Counter(true_labels.tolist()).most_common(1)[0][0]
            cluster_to_label[cluster] = most_common_label

    return cluster_to_label


### Create Centroids

In [105]:
for subset in num_subsets:
    cluster_labels, centroids = kmeans(train_data, subset)
    cluster_to_label = assign_labels(cluster_labels, train_labels, subset)
    torch.save(centroids, f"mnist_kmeans_centroids_{subset}.pth")
    torch.save(cluster_to_label, f"mnist_kmeans_cluster_labels_{subset}.pth")

Converged at iteration 10
Converged at iteration 17
Converged at iteration 51


### Make Predictions

In [123]:
accuracy_dict_kmeans = {}

for subset in num_subsets:
    prototype_train_data = torch.load(f"mnist_kmeans_centroids_{subset}.pth", weights_only=False)
    prototype_train_labels = torch.tensor(torch.load(f"mnist_kmeans_cluster_labels_{subset}.pth", weights_only=False))
    accuracy = compute_accuracy(test_data, prototype_train_data, prototype_train_labels)
    accuracy_dict_kmeans[subset] = [accuracy]

Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9594
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9560
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9408


In [124]:
accuracy_df_kmeans = pd.DataFrame(accuracy_dict_kmeans)
accuracy_df_kmeans

,10000,5000,1000
0,0.9594,0.956,0.9408


## K Means ++

### Base Functions

In [ ]:
def kmeans_plusplus(X, k):
    """
    Initializes centroids using the K-Means++ algorithm.
    
    Args:
    - X (Tensor): Data points of shape (N, D)
    - k (int): Number of clusters
    
    Returns:
    - centroids (Tensor): Initialized centroids of shape (k, D)
    """
    N, D = X.shape
    device = X.device
    centroids = torch.empty((k, D), device=device)  # To store k centroids

    # 1️⃣ Select first centroid randomly
    first_idx = torch.randint(0, N, (1,))
    centroids[0] = X[first_idx]

    # 2️⃣ Select remaining centroids using distance-based probability
    for i in range(1, k):
        distances = torch.cdist(X, centroids[:i]).min(dim=1).values ** 2
        probabilities = distances / distances.sum()  # Normalize to probabilities
        next_idx = torch.multinomial(probabilities, 1)  # Sample next centroid
        centroids[i] = X[next_idx]

    return centroids

def kmeans(X, k, num_iters=100, tol=1e-4):
    """
    Performs K-Means clustering with K-Means++ initialization.
    
    Args:
    - X (Tensor): Data points of shape (N, D)
    - k (int): Number of clusters
    - num_iters (int): Number of iterations
    - tol (float): Tolerance for convergence
    
    Returns:
    - cluster_assignments (Tensor): Cluster indices for each point
    - centroids (Tensor): Final cluster centroids
    """
    N, D = X.shape
    device = X.device

    # Initialize centroids using K-Means++
    centroids = kmeans_plusplus(X, k)

    for _ in range(num_iters):
        # Compute distances from each point to each centroid
        distances = torch.cdist(X, centroids)  # (N, k)

        # Assign each point to the nearest centroid
        cluster_assignments = torch.argmin(distances, dim=1)  # (N,)

        # Compute new centroids
        new_centroids = torch.stack([X[cluster_assignments == c].mean(dim=0) for c in range(k)])

        # Check for convergence (if centroids change very little)
        if torch.allclose(new_centroids, centroids, atol=tol):
            print("Converged early!")
            break

        centroids = new_centroids

    return cluster_assignments, centroids


### Create Centroids

In [128]:
for subset in num_subsets:
    cluster_labels, centroids = kmeans(train_data, subset)
    cluster_to_label = assign_labels(cluster_labels, train_labels, subset)
    torch.save(centroids, f"mnist_kmeans_plus_centroids_{subset}.pth")
    torch.save(cluster_to_label, f"mnist_kmeans_plus_cluster_labels_{subset}.pth")

Converged early!
Converged early!
Converged early!


### Make Predictions

In [131]:
accuracy_dict_kmeans_plus = {}

for subset in num_subsets:
    prototype_train_data = torch.load(f"mnist_kmeans_plus_centroids_{subset}.pth", weights_only=False)
    prototype_train_labels = torch.tensor(torch.load(f"mnist_kmeans_plus_cluster_labels_{subset}.pth", weights_only=False))
    accuracy = compute_accuracy(test_data, prototype_train_data, prototype_train_labels)
    accuracy_dict_kmeans_plus[subset] = [accuracy]

Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9606
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9583
Computing full distance matrix...
k-NN accuracy on full test set (no batching): 0.9427


In [132]:
accuracy_df_kmeans_plus = pd.DataFrame(accuracy_dict_kmeans_plus)
accuracy_df_kmeans_plus

,10000,5000,1000
0,0.9606,0.9583,0.9427


In [134]:
accuracy_df.mean()

10000    0.948923
5000     0.935643
1000     0.883680
dtype: float64

In [136]:
accuracy_df_kmeans

,10000,5000,1000
0,0.9594,0.956,0.9408
